In [1]:
import os
import random
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from data_reader import MaldiDataset
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, balanced_accuracy_score
from preprocessing_components import  ColumnDropper, SpectrumExpander, IntensityScaler, TICLogTransformer
import joblib
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import plotly.express as px
from IPython.display import IFrame
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
from IPython.display import IFrame
import plotly.io as pio
import os

# Carga de datos

In [3]:
# Set all seeds to make the results reproducible
random.seed(42)
np.random.seed(42)


# This script is a simple starting point to classify fungal data using MALDI-TOF spectra.
class SimpleFungusDataLoader:
    def __init__(self, dataset_path, test_size=0.2, random_state=42):
        # Initialize the classifier with dataset path, test size, and random state for reproducibility.
        self.dataset_path = dataset_path
        self.test_size = test_size
        self.random_state = random_state
        self.data = None
        self.train_data = []
        self.test_data = []

    def load_data(self, n_step):
        # Load the dataset using MaldiDataset
        dataset = MaldiDataset(self.dataset_path, n_step=n_step)
        dataset.parse_dataset()  # Parse the dataset from the specified path
        self.data = dataset.get_data()  # Retrieve the parsed data


    def split_data_stratify(self):
        """
        Divide los datos en train y test de forma estratificada según 'genus_species_label',
        asegurando que no haya solapamiento de 'unique_id_label' entre ambos conjuntos.
        Las clases con menos de 2 instancias se asignan directamente al conjunto de entrenamiento.
        """
        # Convertir los datos en un DataFrame
        df = pd.DataFrame(self.data)

        # Agrupar por 'unique_id_label' y seleccionar una clase representativa ('genus_species_label') para cada grupo
        unique_id_groups = df.groupby('unique_id_label').first().reset_index()

        # Identificar las clases con menos de 2 instancias
        class_counts = unique_id_groups['genus_species_label'].value_counts()
        small_classes = class_counts[class_counts < 2].index

        # Separar los grupos con clases pequeñas y el resto
        small_class_groups = unique_id_groups[unique_id_groups['genus_species_label'].isin(small_classes)]
        remaining_groups = unique_id_groups[~unique_id_groups['genus_species_label'].isin(small_classes)]

        # Estratificar las clases restantes
        train_ids, test_ids = train_test_split(
            remaining_groups['unique_id_label'],
            test_size=self.test_size,
            random_state=self.random_state,
            stratify=remaining_groups['genus_species_label']  # Usar 'genus_species_label' como criterio de estratificación
        )

        # Agregar todas las instancias de clases pequeñas al conjunto de entrenamiento
        train_ids = pd.concat([pd.Series(train_ids), small_class_groups['unique_id_label']])

        # Filtrar el DataFrame original para crear los conjuntos de train y test
        self.train_data = df[df['unique_id_label'].isin(train_ids)]  # DataFrame de entrenamiento
        self.test_data = df[df['unique_id_label'].isin(test_ids)]  # DataFrame de prueba

        # Verificar que no haya solapamiento de 'unique_id_label' entre train y test
        train_unique_ids = set(self.train_data['unique_id_label'])
        test_unique_ids = set(self.test_data['unique_id_label'])
        assert len(train_unique_ids.intersection(test_unique_ids)) == 0, "Unique ID labels overlap between train and test"

        # Imprimir estadísticas
        print(f"Number of unique_id_labels in train data: {len(train_unique_ids)}")
        print(f"Number of unique_id_labels in test data: {len(test_unique_ids)}")
        print(f"Number of samples in train data: {len(self.train_data)}")
        print(f"Number of samples in test data: {len(self.test_data)}")
        print(f"Number of classes to predict: {len(self.train_data['genus_species_label'].unique())}")



    def plot_data_distribution(self):
        """
        Grafica la distribución de las clases ('genus_species_label') en los conjuntos de
        entrenamiento y prueba para visualizar las proporciones después de la división estratificada.
        """
        # Contar las etiquetas en los conjuntos de entrenamiento y prueba
        train_counts = self.train_data['genus_species_label'].value_counts()
        test_counts = self.test_data['genus_species_label'].value_counts()

        # Unificar las etiquetas para asegurar que ambas series tengan los mismos índices
        all_labels = pd.Index(train_counts.index).union(test_counts.index)
        train_counts = train_counts.reindex(all_labels, fill_value=0)
        test_counts = test_counts.reindex(all_labels, fill_value=0)

        # Crear gráfico
        x = np.arange(len(all_labels))  # Posiciones de las barras
        width = 0.4  # Ancho de las barras

        plt.figure(figsize=(14, 8))  # Tamaño del gráfico
        plt.bar(x - width / 2, train_counts, width, label='Train', alpha=0.8, color='blue')
        plt.bar(x + width / 2, test_counts, width, label='Test', alpha=0.8, color='orange')

        # Configurar etiquetas y título
        plt.xlabel('Genus+Species Label', fontsize=12)
        plt.ylabel('Number of Samples', fontsize=12)
        plt.title('Distribution of Genus+Species Labels in Train and Test Data', fontsize=14)
        plt.xticks(x, all_labels, rotation=90, fontsize=10)  # Etiquetas en el eje X
        plt.legend(fontsize=12)

        # Ajustar diseño
        plt.tight_layout()
        plt.grid(axis='y', linestyle='--', alpha=0.6)
    
    def get_train_data(self):
        return self.train_data
    
    def get_test_data(self):
        return self.test_data
    

        


In [4]:
# Define the dataset path (update this path to where your dataset is located)
dataset_path = "data/fungus_db"

# Initialize the classifier with the dataset path
fungus_identifier = SimpleFungusDataLoader(dataset_path)

fungus_identifier.load_data(n_step=3)

# Load and split the data into training and test sets.
fungus_identifier.split_data_stratify()

c:\Users\aleta\Documents\TFM\FungusClassifier\data_reader.py:299: RuntimeWarning: invalid value encountered in divide
  intensity=SpectrumObj.intensity / SpectrumObj.intensity.sum() * self.sum,


Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Skipping nan spectrum
Number of unique_id_labels in train data: 268
Number of unique_id_labels in test data: 64
Number of samples in train data: 6452
Number of samples in test data: 1476
Number of classes to predict: 60


# Cargando pipeline de preprocesado

In [8]:
preproc = joblib.load('preproc.pkl')

In [9]:
label_encoder_genus = LabelEncoder()
label_encoder_species = LabelEncoder()
x_train = fungus_identifier.get_train_data()
X = x_train.drop(columns=['genus_species_label', 'genus_label'])
y_species = label_encoder_species.fit_transform(x_train['genus_species_label'])
y_genus = label_encoder_genus.fit_transform(x_train['genus_label'])

# HSIC Lasso

# Modelos clásicos

In [10]:
def evaluate_with_cross_validation(pipe, X_df, y, group_col='unique_id_label',
                                         n_splits=6, random_state=42):
    """
    Evalúa un pipeline con GroupKFold, calculando métricas globales y por clase,
    asegurando que todas las muestras con el mismo group_col queden en el mismo fold.
    
    Parámetros
    ----------
    pipe        : Pipeline de sklearn (preproc + clf)
    X_df        : pandas.DataFrame con las features crudas _incluyendo_ group_col
    y           : array‑like 1D de etiquetas codificadas
    group_col   : str, nombre de la columna en X_df que contiene los grupos
    n_splits    : int, número de folds
    random_state: semilla para reproducibilidad (solo afecta al ordering interno)
    
    Devuelve
    -------
    df_global   : DataFrame con métricas globales por fold
    df_class    : DataFrame con métricas por clase por fold
    """
    groups = X_df[group_col].values
    cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    global_metrics = []
    per_class      = []
    
    for fold, (train_idx, test_idx) in enumerate(cv.split(X_df, y, groups), start=1):
        # 1) Chequeo de índices
        assert set(train_idx).isdisjoint(test_idx), \
            f"Fold {fold}: ¡solapamiento de índices!"
        
        # 2) Extraer splits
        X_train, X_test = X_df.iloc[train_idx], X_df.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # 3) Ajustar pipeline y predecir
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        
        # 4) Métricas globales
        global_metrics.append({
            "fold":             fold,
            "accuracy":         accuracy_score(y_test, y_pred),
            "precision_macro":  precision_score(y_test, y_pred, average="macro", zero_division=0),
            "recall_macro":     recall_score(y_test, y_pred, average="macro", zero_division=0),
            "f1_macro":         f1_score(y_test, y_pred, average="macro", zero_division=0),
            "balanced_accuracy": balanced_accuracy_score(y_test, y_pred),
        })
        
        # 5) Métricas por clase
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        for cls, m in report.items():
            if cls in ("accuracy", "macro avg", "weighted avg"):
                continue
            per_class.append({
                "fold":     fold,
                "class":    cls,
                "precision": m["precision"],
                "recall":    m["recall"],
                "f1":        m["f1-score"],
                "support":   m["support"]
            })
        
        # 6) Mostrar informe de texto de este fold
        print(f"\n--- Fold {fold} (Groups) ---")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    # 7) DataFrames de salida
    df_global = pd.DataFrame(global_metrics).set_index("fold")
    df_class  = pd.DataFrame(per_class)
    
    # 8) Resúmenes finales
    print("\nMétricas globales por fold:")
    print(df_global)
    print("\nResumen global (media ± std):")
    print(df_global.agg(["mean","std"]))
    print("\nMétricas por clase (media ± std sobre folds):")
    print(
        df_class
          .groupby("class")[["precision","recall","f1"]]
          .agg(["mean","std"])
    )
    
    return df_global, df_class


## KNN

### Aleatorio

In [12]:
knn_classifier = KNeighborsClassifier()
knn_pipeline = Pipeline([
    ('preproc', preproc),
    ('clf', knn_classifier)
]
)
x_df = X.copy()
evaluate_with_cross_validation(knn_pipeline, x_df, y_species, n_splits=4, random_state=42)

c:\Users\aleta\anaconda3\envs\TFM\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")



--- Fold 1 (Groups) ---
              precision    recall  f1-score   support

           1       1.00      0.38      0.55        71
           3       0.94      1.00      0.97        89
           4       0.00      0.00      0.00         0
           6       1.00      1.00      1.00        47
           7       1.00      1.00      1.00       101
           9       0.46      0.51      0.49        51
          10       0.35      1.00      0.52        24
          12       0.90      0.76      0.83        93
          14       0.00      0.00      0.00        20
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00      0.00      0.00        20
          22       1.00      1.00      1.00        48
          23       0.90      1.00      0.95        99
          25       0.59      0.55      0.57        66
          26       1.00      1.00      1.00        44
          31       0.00      0.00      0.00        20
  

c:\Users\aleta\anaconda3\envs\TFM\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")



--- Fold 2 (Groups) ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       0.36      0.56      0.44        43
           3       0.93      1.00      0.96        64
           4       0.00      0.00      0.00        27
           5       0.77      1.00      0.87        23
           6       0.96      1.00      0.98        25
           7       0.79      1.00      0.88       140
           8       0.00      0.00      0.00        27
           9       0.98      0.73      0.84        64
          10       0.54      0.34      0.42        64
          11       0.00      0.00      0.00        24
          12       0.51      1.00      0.68        53
          13       0.00      0.00      0.00        25
          17       1.00      1.00      1.00        24
          18       1.00      1.00      1.00        45
          19       1.00      1.00      1.00        30
          20       0.00      0.00      0.00         0
  

c:\Users\aleta\anaconda3\envs\TFM\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")



--- Fold 3 (Groups) ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       0.52      1.00      0.69        24
           2       0.00      0.00      0.00        22
           3       0.27      1.00      0.42        26
           4       0.00      0.00      0.00         0
           6       0.98      1.00      0.99        98
           7       0.71      0.96      0.82        49
           9       0.51      1.00      0.67        45
          10       1.00      0.75      0.86        87
          12       0.54      1.00      0.70        27
          15       0.00      0.00      0.00        40
          19       1.00      1.00      1.00        26
          20       0.00      0.00      0.00        23
          22       1.00      1.00      1.00        20
          23       0.75      1.00      0.86        83
          24       1.00      1.00      1.00        54
          25       1.00      0.49      0.66        43
  

c:\Users\aleta\anaconda3\envs\TFM\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


(      accuracy  precision_macro  recall_macro  f1_macro  balanced_accuracy
 fold                                                                      
 1     0.762524         0.597668      0.571751  0.559749           0.696822
 2     0.813033         0.729692      0.758784  0.733033           0.796723
 3     0.705989         0.520547      0.576241  0.524155           0.682390
 4     0.824691         0.687996      0.705001  0.677483           0.781217,
      fold class  precision    recall        f1  support
 0       1     1   1.000000  0.380282  0.551020     71.0
 1       1     3   0.936842  1.000000  0.967391     89.0
 2       1     4   0.000000  0.000000  0.000000      0.0
 3       1     6   1.000000  1.000000  1.000000     47.0
 4       1     7   1.000000  1.000000  1.000000    101.0
 ..    ...   ...        ...       ...       ...      ...
 162     4    54   1.000000  1.000000  1.000000     24.0
 163     4    56   0.909091  1.000000  0.952381     70.0
 164     4    57   1.000000  1

In [13]:
if not os.path.exists("bin_3_models_pipelines"):
    os.makedirs("bin_3_models_pipelines")

joblib.dump(knn_pipeline, "./bin_3_models_pipelines/3_bin_knn_pipeline.pkl")

['./bin_3_models_pipelines/3_bin_knn_pipeline.pkl']

### GridSearch

In [43]:
x_proc = preproc.fit_transform(X.copy())

In [54]:
from sklearn.model_selection import GridSearchCV

knn_classifier = KNeighborsClassifier()

groups = X['unique_id_label'].values
cv = GroupKFold(n_splits=6)

param_grid = {
    'n_neighbors': [3, 5, 10],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size' : [30, 40, 50]
}

grid_search = GridSearchCV(knn_classifier, param_grid, cv=cv.split(x_proc, y_species, groups), scoring='balanced_accuracy', n_jobs=1, verbose=3)

In [ ]:
grid_search.fit(x_proc, y_species)

In [56]:
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f'{params}:   {mean_score}')

{'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 3, 'weights': 'uniform'}:   0.7564179126005662
{'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 3, 'weights': 'distance'}:   0.7573479744520366
{'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 5, 'weights': 'uniform'}:   0.7512066215186435
{'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 5, 'weights': 'distance'}:   0.7532055620058609
{'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 10, 'weights': 'uniform'}:   0.7476851057948172
{'algorithm': 'auto', 'leaf_size': 30, 'n_neighbors': 10, 'weights': 'distance'}:   0.7501952985249378
{'algorithm': 'auto', 'leaf_size': 40, 'n_neighbors': 3, 'weights': 'uniform'}:   0.7564179126005662
{'algorithm': 'auto', 'leaf_size': 40, 'n_neighbors': 3, 'weights': 'distance'}:   0.7573479744520366
{'algorithm': 'auto', 'leaf_size': 40, 'n_neighbors': 5, 'weights': 'uniform'}:   0.7512066215186435
{'algorithm': 'auto', 'leaf_size': 40, 'n_neighbors': 5, 'weights': 'distance'}:   0.

# Random Forest

## Aleatorio

In [37]:
rf_classifier = RandomForestClassifier()
rf_pipeline = Pipeline([
    ('preproc', preproc),
    ('clf', rf_classifier)
]
)

In [38]:
x_df = X.copy()
evaluate_with_cross_validation(rf_pipeline, x_df, y_species, n_splits=3, random_state=42)

c:\Users\aleta\anaconda3\envs\TFM\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning:

y_pred contains classes not in y_true




--- Fold 1 (Groups) ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       0.31      0.57      0.40        51
           3       0.55      1.00      0.71        69
           4       0.00      0.00      0.00        27
           5       0.00      0.00      0.00         0
           6       0.99      1.00      0.99        76
           7       1.00      1.00      1.00       195
           8       0.00      0.00      0.00         0
           9       0.73      0.65      0.69        69
          10       0.67      0.41      0.51       109
          12       0.72      0.98      0.83       106
          14       0.00      0.00      0.00        20
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00        22
          19       0.00      0.00      0.00        56
          20       0.00      0.00      0.00         0
          21       0.00      0.00      0.00         0
  

c:\Users\aleta\anaconda3\envs\TFM\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning:

y_pred contains classes not in y_true




--- Fold 2 (Groups) ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        44
           1       0.98      0.48      0.65        89
           2       0.00      0.00      0.00        22
           3       0.98      0.99      0.99       158
           4       0.00      0.00      0.00         0
           6       1.00      1.00      1.00        76
           7       0.77      1.00      0.87       169
           8       0.00      0.00      0.00        27
           9       0.66      1.00      0.79        73
          10       0.30      1.00      0.46        20
          11       0.00      0.00      0.00        24
          12       0.60      0.53      0.57        73
          15       0.00      0.00      0.00         0
          17       0.00      0.00      0.00         0
          18       1.00      0.13      0.24        45
          19       0.00      0.00      0.00         0
          20       0.00      0.00      0.00        23
  

c:\Users\aleta\anaconda3\envs\TFM\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning:

y_pred contains classes not in y_true



(      accuracy  precision_macro  recall_macro  f1_macro  balanced_accuracy
 fold                                                                      
 1     0.729050         0.488267      0.540229  0.495812           0.632463
 2     0.808530         0.646664      0.607583  0.597418           0.713910
 3     0.713554         0.473088      0.561080  0.502891           0.626322,
      fold class  precision    recall        f1  support
 0       1     0   1.000000  1.000000  1.000000     20.0
 1       1     1   0.311828  0.568627  0.402778     51.0
 2       1     3   0.552000  1.000000  0.711340     69.0
 3       1     4   0.000000  0.000000  0.000000     27.0
 4       1     5   0.000000  0.000000  0.000000      0.0
 ..    ...   ...        ...       ...       ...      ...
 138     3    52   1.000000  1.000000  1.000000     84.0
 139     3    53   1.000000  1.000000  1.000000     52.0
 140     3    56   0.783333  1.000000  0.878505     94.0
 141     3    58   1.000000  1.000000  1.000000  

In [30]:
if not os.path.exists("bin_3_models_pipelines"):
    os.makedirs("bin_3_models_pipelines")

joblib.dump(rf_pipeline, "./bin_3_models_pipelines/3_bin_rf_pipeline.pkl")

['./bin_3_models_pipelines/3_bin_rf_pipeline.pkl']

## SVM

In [14]:
svm_classifier = SVC(kernel = 'rbf')
svm_pipeline = Pipeline([
    ('preproc', preproc),
    ('clf', svm_classifier)
]
)
x_df = X.copy()

In [ ]:
evaluate_with_cross_validation(svm_pipeline, x_df, y_species, n_splits=3, random_state=42)

# PCA para visualización 3D

In [ ]:

x_df = X.copy()
# 1) Preprocesado de features usando tu pipeline
X_proc = preproc.fit_transform(x_df)

# 2) PCA a 3 componentes
pca = PCA(n_components=3, random_state=42)
X_pca = pca.fit_transform(X_proc)

# 3) DataFrame para graficar
df_vis = pd.DataFrame(X_pca, columns=['PC1','PC2','PC3'])
df_vis['genus']   = label_encoder_genus.inverse_transform(y_genus)
df_vis['species'] = label_encoder_species.inverse_transform(y_species)

# 4) Grafico 3D interactivo – Género
fig_genus = px.scatter_3d(
    df_vis, x='PC1', y='PC2', z='PC3',
    color='genus',
    title='PCA 3D interactivo – Género',
    labels={'genus':'Género'}
)

if not os.path.exists("bin_3_pca"):
    os.makedirs("bin_3_pca")

# 5) O si prefieres guardarlo como HTML y luego incrustarlo:
html_path = "./bin_3_pca/bin_3_pca_genus.html"

display(IFrame(html_path, width=800, height=600))

# 6) Mismo para Especie
fig_species = px.scatter_3d(
    df_vis, x='PC1', y='PC2', z='PC3',
    color='species',
    title='PCA 3D interactivo – Especie',
    labels={'species':'Especie'}
)

html_path2 = "./bin_3_pca/bin_3_pca_species.html"


fig_species.write_html(html_path2, include_plotlyjs='cdn')
fig_genus.write_html(html_path, include_plotlyjs='cdn')


# TSNE3 para visualización 3D

In [ ]:
# 0) Forzamos renderer interactivo
pio.renderers.default = "iframe_connected"

# 1) Creamos directorio si no existe
if not os.path.exists("bin_3_tsne3"):
    os.makedirs("bin_3_tsne3")
    
x_df = X.copy()

# 2) Preprocesado de features usando tu pipeline
X_proc = preproc.fit_transform(x_df)

# 3) t‑SNE a 3 componentes
tsne = TSNE(n_components=3, random_state=42, init='pca', learning_rate='auto')
X_tsne = tsne.fit_transform(X_proc)

# 4) DataFrame para graficar
df_vis = pd.DataFrame(X_tsne, columns=['TSNE1','TSNE2','TSNE3'])
df_vis['genus']   = label_encoder_genus.inverse_transform(y_genus)
df_vis['species'] = label_encoder_species.inverse_transform(y_species)

# 5) Gráfico 3D interactivo – Género
fig_genus = px.scatter_3d(
    df_vis,
    x='TSNE1', y='TSNE2', z='TSNE3',
    color='genus',
    title='t-SNE 3D interactivo – Género',
    labels={'genus':'Género'}
)
html_path = "./bin_3_tsne3/tsne3_genus.html"
fig_genus.write_html(html_path, include_plotlyjs='cdn')

# 6) Gráfico 3D interactivo – Especie
fig_species = px.scatter_3d(
    df_vis,
    x='TSNE1', y='TSNE2', z='TSNE3',
    color='species',
    title='t-SNE 3D interactivo – Especie',
    labels={'species':'Especie'}
)
html_path2 = "./bin_3_tsne3/tsne3_species.html"
fig_species.write_html(html_path2, include_plotlyjs='cdn')
